# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

In [18]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE = "http://books.toscrape.com/"
CATALOGUE = urljoin(BASE, "catalogue/")

HEADERS = {"User-Agent": "Mozilla/5.0"}
TIMEOUT = 10

def fetch_soup(url):
    r = requests.get(url, headers=HEADERS, timeout=TIMEOUT)
    r.raise_for_status()
    return BeautifulSoup(r.content, "html.parser")

def get_book_url():
    all_book_url = []
    for number in range(1, 51):
        page_url = urljoin(CATALOGUE, f"page-{number}.html")
        try:
            soup = fetch_soup(page_url)
        except requests.RequestException as e:
            print(f"page not found {page_url} -> {e}")
            continue

        for a in soup.select("article.product_pod h3 a"):
            href = a.get("href")
           
            book_url = urljoin(page_url, href)
            all_book_url.append(book_url)

    return all_book_url

def text_or_none(el):
    return el.get_text(strip=True) if el else None

def parse_price(text):
    
    if not text:
        return None
    m = re.search(r"[\d.]+", text)
    return float(m.group()) if m else None

def get_book_data(min_rating, max_price):
    all_book_url = get_book_url()
    word_to_num = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}

    rows = []

    for url in all_book_url:
        try:
            soup = fetch_soup(url)
        except requests.RequestException:
            continue

        
        title = text_or_none(soup.select_one("div.product_main h1"))

        
        rating_word = None
        rating_p = soup.select_one("div.product_main p.star-rating")
        if rating_p and "class" in rating_p.attrs and len(rating_p["class"]) > 1:
            rating_word = rating_p["class"][1]
        rating_num = word_to_num.get(rating_word, 0)

        
        table = {row.th.get_text(strip=True): row.td.get_text(strip=True)
                 for row in soup.select("table.table.table-striped tr")}
        upc = table.get("UPC")
        price_incl = table.get("Price (incl. tax)")
        availability = table.get("Availability")

        price_val = parse_price(price_incl)

        
        crumbs = [c.get_text(strip=True) for c in soup.select("ul.breadcrumb li a")]
        genre = crumbs[2] if len(crumbs) >= 3 else None

       
        desc_header = soup.select_one("#product_description")
        if desc_header:
            desc = desc_header.find_next("p")
            description = text_or_none(desc)
        else:
            description = None

        
        if (price_val is not None) and (rating_num is not None):
            if (price_val <= max_price) and (rating_num >= min_rating):
                rows.append({
                    "title": title,
                    "upc": upc,
                    "price_taxIncluded": price_incl,
                    "rating": rating_word,
                    "genre": genre,
                    "availability": availability,
                    "description": description,
                    "url": url
                })

    df = pd.DataFrame(rows, columns=[
        "title","upc","price_taxIncluded","rating","genre","availability","description","url"
    ])
    return df



In [19]:
df_books_lowPrice_highRating

,title,upc,price_taxIncluded,rating,genre,availability,description
12,Set Me Free,ce6396b0f23f6ecc,£17.46,Five,Young Adult,In stock (19 available),Aaron Ledbetter’s future had been planned out ...
30,The Four Agreements: A Practical Guide to Pers...,6258a1f6a6dcfe50,£17.66,Five,Spirituality,In stock (18 available),"In The Four Agreements, don Miguel Ruiz reveal..."
34,Sophie's World,6be3beb0793a53e7,£15.94,Five,Philosophy,In stock (18 available),A page-turning novel that is also an explorati...
47,Untitled Collection: Sabbath Poems 2014,657fe5ead67a7767,£14.27,Four,Poetry,In stock (16 available),"More than thirty-five years ago, when the weat..."
53,This One Summer,51653ef291ab7ddc,£19.49,Four,Sequential Art,In stock (16 available),"Every summer, Rose goes with her mom and dad t..."
...,...,...,...,...,...,...,...
913,The Zombie Room,9c96cd1329fbd82d,£19.69,Five,Default,In stock (1 available),An unlikely bond is forged between three men f...
917,The Silent Wife,b78deb463531d078,£12.34,Five,Fiction,In stock (1 available),A chilling psychological thriller about a marr...
934,The Girl You Lost,4280ac3eab57aa5d,£12.29,Five,Mystery,In stock (1 available),Eighteen years ago your baby daughter was snat...
937,The Edge of Reason (Bridget Jones #2),29fc016c459aeb14,£19.18,Four,Womens Fiction,In stock (1 available),Monday 27 January“7:15 a.m. Hurrah! The wilder...
